# 1. Problem Statement
The project is aimed to generate images in the style of Monet. At least 7,000 to 10,000 Monet-style images is going to be generated. The generator will be a neural network that creates the images. This generator is trained using a discriminator. Meanwhile, a discriminator is trying to accurately classify the real or generated images. A GAN architecture is going to be used for that two neural networks: generator and discriminator. 

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

from kaggle_datasets import KaggleDatasets
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
from PIL import Image

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

AUTOTUNE = tf.data.experimental.AUTOTUNE
    
print(tf.__version__)

# 2. Exploratory Data Analysis (EDA)
All the images are already 256x256 pixels and RGB images.

To train the model, scale the images to a [-1, 1] scale. 

In [ ]:
import matplotlib.image as mpimg
import os

#get image file name and dir
dir_images_monet = '/kaggle/input/monet-gan-getting-started/monet_jpg/'
image_monet=[]
for dirname, _, filenames in os.walk(dir_images_monet):
    for filename in filenames:
        image_monet.append(os.path.join(dirname, filename))

#get image info
monetData = Image.open(image_monet[0])
print("Image Meta Data:", monetData)
monetData.info
monet = mpimg.imread(image_monet[0])
plt.imshow(monet)





In [ ]:
dir_images_photo = '/kaggle/input/monet-gan-getting-started/photo_jpg/'
image_photo=[]
for dirname, _, filenames in os.walk(dir_images_photo):
    for filename in filenames:
        image_photo.append(os.path.join(dirname, filename))
        
photoData = Image.open(image_photo[0])
print("Image Meta Data:", photoData)
photoData.info
photo = mpimg.imread(image_photo[0])
plt.imshow(photo)

In [ ]:
IMAGE_SIZE = [256, 256]

def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = (tf.cast(image, tf.float32) / 127.5) - 1
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

def read_tfrecord(example):
    tfrecord_format = {
        "image_name": tf.io.FixedLenFeature([], tf.string),
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    return image

def load_dataset(filenames, labeled=True, ordered=False):
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTOTUNE)
    return dataset

In [ ]:
monet_ds = load_dataset(MONET_FILENAMES, labeled=True).batch(1)
photo_ds = load_dataset(PHOTO_FILENAMES, labeled=True).batch(1)

In [ ]:
example_monet = next(iter(monet_ds))
example_photo = next(iter(photo_ds))

We could see the sample images. 

In [ ]:
plt.subplot(121)
plt.title('Photo')
plt.imshow(example_photo[0] * 0.5 + 0.5)

plt.subplot(122)
plt.title('Monet')
plt.imshow(example_monet[0] * 0.5 + 0.5)